In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [4]:
import keras
import tensorflow as tf

#tf.enable_eager_execution()

In [5]:
seed_value = 0

np.random.seed(seed_value)
tf.set_random_seed(seed_value)

# 1 data&feat

In [6]:
tf.keras.preprocessing.sequence?

In [7]:
tf.data.TextLineDataset?

In [8]:
tf.data.Dataset.concatenate?

In [9]:
tf.data.experimental.cardinality?

In [10]:
import os

## 1.1 load data

In [11]:
train_file_names = [[os.path.join('../data/aclImdb/train/pos/',i) for i in os.listdir('../data/aclImdb/train/pos/')]
             ,[os.path.join('../data/aclImdb/train/neg/',i) for i in os.listdir('../data/aclImdb/train/neg/')]]
test_file_names=[[os.path.join('../data/aclImdb/test/pos/',i) for i in os.listdir('../data/aclImdb/test/pos/')]
             ,[os.path.join('../data/aclImdb/test/neg/',i) for i in os.listdir('../data/aclImdb/test/neg/')]]

In [16]:
train_text_data = list()
train_text_label = list()
for i,j in enumerate(train_file_names):
    label = 1-i%2
    for file in j:
        file_data = open(file,'r').read()
        train_text_data.append(file_data)
        train_text_label.append(label)

In [17]:
train_file_names[0][0]

'../data/aclImdb/train/pos/4715_9.txt'

In [18]:
test_text_data = list()
test_text_label = list()
for i,j in enumerate(test_file_names):
    label = 1-i%2
    for file in j:
        file_data = open(file,'r').read()
        test_text_data.append(file_data)
        test_text_label.append(label)

In [21]:
test_text_label[0]

1

In [39]:
train_data = list(zip(train_text_data,train_text_label))

In [40]:
np.random.shuffle(train_data)

In [41]:
list(train_data)[:2]

[("It took 9 years to complete this film. I would think that within those 9 years someone would have said,hey, this film is terrible. I've seen better acting in porn movies. The story is tired and played. Abused child turns into serial killer. How about something new for a change. How about abused child turns into a florist? At least that would have been a new twist. Why is it that everyone with a camera and a movie idea (especially unoriginal movie ideas) thinks that they can be a director? I do admire the fact that they stuck with this film for 9 years to get it completed. That shows tenacity and spirit. With this kind of drive hopefully next time they can focus it on a better script. If you want to see a failed experiment in indie film making from a writer/director from Michigan see Hatred of A Minute. If you want a good movie from a Michigan writer/director stick with Evil Dead.",
  0),
 ("Stan & Ollie become SAPS AT SEA when their wayward little boat is commandeered by a vicious m

In [43]:
train_data_eval = train_data[:int(np.array(train_data).shape[0]*0.2)]
train_data_train = train_data[int(np.array(train_data).shape[0]*0.2):]

In [52]:
train_data_x = [i[0] for i in train_data_train]
train_data_y = [i[1] for i in train_data_train]

eval_data_x = [i[0] for i in train_data_eval]
eval_data_y = [i[1] for i in train_data_eval]

In [74]:
test_data_y = test_text_label

In [53]:
train_data_x[:2]

['Sisters in law will be released theatrically on march 24th in Sweden. A good occasion for our Nordic friends to discover this original and thoughtful documentary. It was shown in Göteborg together with a retrospective dedicated to Kim Longinotto, "director in focus" of the festival. She gave a master class, very much appreciated, telling about her method as documentary filmmaker and told the audience about the special circumstances which led her to shoot Sisters in law twice : the first version got lost for good, so a second shooting was organized and the film turned out to be different at the end. A pretty awful problem happened, in this case, to create the possibility of a very strong movie.',
 'In a year of pretentious muck like "Synecdoche, New York" a film born out of Charlie Kaufman\'s own self-indulgence, comes a film that is similarly hard to watch but about three times as important. "Frownland" is a labor of love by the crew, the actors and the filmmaker, shot over years by 

In [54]:
eval_data_x[:2]

["It took 9 years to complete this film. I would think that within those 9 years someone would have said,hey, this film is terrible. I've seen better acting in porn movies. The story is tired and played. Abused child turns into serial killer. How about something new for a change. How about abused child turns into a florist? At least that would have been a new twist. Why is it that everyone with a camera and a movie idea (especially unoriginal movie ideas) thinks that they can be a director? I do admire the fact that they stuck with this film for 9 years to get it completed. That shows tenacity and spirit. With this kind of drive hopefully next time they can focus it on a better script. If you want to see a failed experiment in indie film making from a writer/director from Michigan see Hatred of A Minute. If you want a good movie from a Michigan writer/director stick with Evil Dead.",
 "Stan & Ollie become SAPS AT SEA when their wayward little boat is commandeered by a vicious murderer.

## 1.2 分词& padding example

In [244]:
keras.preprocessing.text.Tokenizer?

In [66]:
keras.preprocessing.text.Tokenizer.fit_on_sequences?

In [62]:
token = keras.preprocessing.text.Tokenizer(num_words=20000,oov_token=True,split=' ')

In [63]:
token.fit_on_texts(train_data_x)

In [64]:
type(train_data_x)

list

In [69]:
train_data_x = token.texts_to_sequences(train_data_x)
eval_data_x = token.texts_to_sequences(eval_data_x)
test_data_x = token.texts_to_sequences(test_text_data)

In [71]:
keras.preprocessing.sequence.pad_sequences?

In [72]:
MAX_LEN = 500

In [73]:
train_data_x = keras.preprocessing.sequence.pad_sequences(train_data_x,maxlen=MAX_LEN,padding='post')
eval_data_x = keras.preprocessing.sequence.pad_sequences(eval_data_x,maxlen=MAX_LEN,padding='post')
test_data_x = keras.preprocessing.sequence.pad_sequences(test_data_x,maxlen=MAX_LEN,padding='post')

In [78]:
tf.data.Dataset.batch?

In [104]:
tf.data.Dataset.shuffle?

In [258]:
train_ds = tf.data.Dataset.from_tensor_slices((train_data_x,train_data_y)).shuffle(buffer_size=20000).cache().batch(128)

eval_ds = tf.data.Dataset.from_tensor_slices((eval_data_x,eval_data_y)).shuffle(buffer_size=5000).cache().batch(128)

test_ds = tf.data.Dataset.from_tensor_slices((test_data_x,test_data_y)).shuffle(buffer_size=25000).cache().batch(256)

# 2 model

In [79]:
##input: batch_size max_len 

In [260]:
from keras.models import Model
from keras.layers import  Input,Embedding,Dense,LSTM,Bidirectional,Conv1D,GlobalAvgPool1D,GlobalMaxPool1D

In [261]:
Embedding?

In [262]:
Conv1D?

In [263]:
GlobalAvgPool1D?

In [264]:
GlobalMaxPool1D?

In [273]:
input_x = tf.keras.layers.Input(shape=(MAX_LEN,),name='input_x')

In [274]:
embed_x = tf.keras.layers.Embedding(input_dim=20000,output_dim=128)(input_x)
embed_x = tf.keras.layers.Dropout(0.5)(embed_x)

In [275]:
x = tf.keras.layers.Conv1D(filters=128,kernel_size=7,padding='valid',activation='relu',strides=3)(embed_x)
x = tf.keras.layers.Conv1D(filters=128,kernel_size=7,padding='valid',activation='relu',strides=3)(x)

In [276]:
x = tf.keras.layers.GlobalMaxPool1D()(x)

In [277]:
pred = tf.keras.layers.Dense(units=1,activation='sigmoid',name='pred')(x)

In [278]:
nn = tf.keras.models.Model(inputs=input_x, outputs=pred)

In [279]:
nn.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_x (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 500, 128)          2560000   
_________________________________________________________________
dropout (Dropout)            (None, 500, 128)          0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 165, 128)          114816    
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 53, 128)           114816    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
pred (Dense)                 (None, 1)                 129

In [280]:
#nn.compile(optimizer='adam',loss=keras.losses.binary_crossentropy)#,metrics=[keras.metrics.binary_accuracy,keras.metrics.binary_crossentropy])

In [281]:
#history = nn.fit(x=train_data_x,y=train_data_y,validation_data=[eval_data_x,eval_data_y],epochs=3)

In [282]:
nn.compile(optimizer='adam',loss=tf.keras.losses.mean_squared_error,metrics=[tf.keras.metrics.binary_accuracy,tf.keras.metrics.binary_crossentropy])

In [283]:
history = nn.fit(train_ds,epochs=3,validation_data=eval_ds)#,steps_per_epoch=int(20000/128),validation_steps=int(5000/128100))

Train on 157 steps, validate on 40 steps
Epoch 1/3
157/157 [==============================] - 514s 3s/step - loss: 0.1862 - binary_accuracy: 0.6858 - binary_crossentropy: 0.5435 - val_loss: 0.1053 - val_binary_accuracy: 0.8572 - val_binary_crossentropy: 0.3369
Epoch 2/3
157/157 [==============================] - 437s 3s/step - loss: 0.0768 - binary_accuracy: 0.8964 - binary_crossentropy: 0.2584 - val_loss: 0.0944 - val_binary_accuracy: 0.8788 - val_binary_crossentropy: 0.3180
Epoch 3/3
157/157 [==============================] - 447s 3s/step - loss: 0.0429 - binary_accuracy: 0.9462 - binary_crossentropy: 0.1611 - val_loss: 0.1062 - val_binary_accuracy: 0.8648 - val_binary_crossentropy: 0.3945


In [ ]:
nn.evaluate(test_ds)

91/98 [==========================>...] - ETA: 9s - loss: 0.1096 - binary_accuracy: 0.8554 - binary_crossentropy: 0.3979 